# Scrape vintage years

Scrape known vintage years for anticipated sentiment analysis as a function of year.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
req = requests.get('http://www.erobertparker.com/newsearch/vintagechart1.aspx/VintageChart.aspx')

In [3]:
soup = BeautifulSoup(req.text, 'lxml')

In [4]:
charts = soup.find_all(attrs={'class':'chart'})

Get the labels as a dataframe.

In [5]:
labels = charts[0].find_all('tr')[1:-1]

label_list = [[(y.find('img'), y.get('rowspan')) 
               for y in x.find_all('td')] 
              for x in labels]

label_len = len(label_list)
empty_list = [np.NaN]*label_len
label_df = pd.DataFrame({'loc1':empty_list, 'loc2':empty_list, 'loc3':empty_list})

for col in range(3):
    pos = 0
    while pos < label_len:

        label = label_list[pos].pop(0)
        
        try:
            text = label[0]
            if text is None:
                text = np.NaN
            else:
                text = text.get('alt')
            
            nrows = label[1]
            if nrows is None:
                nrows = 1
            else:
                nrows = int(nrows)
            
            label_df.loc[pos:pos+nrows, 'loc'+str(col+1)] = text
            pos += nrows
        except:
            pos += 1

In [6]:
label_df.head()

,loc1,loc2,loc3
0,Europe,NaN,"Austria Riesling, Gruner Veltliner"
1,Europe,France,Alsace
2,Europe,France,Bordeaux: St. Julien/Pauillac St. Estephe
3,Europe,France,Margaux
4,Europe,France,Graves


In [7]:
label_df.shape

(51, 3)

Get the year labels for the chart.

In [8]:
year_list = [x.text.strip() for x in charts[1].find('tr').find_all('th')]

In [9]:
len(year_list)

41

Get the rankings for the chart and clean up the values.

In [10]:
ranking_df = pd.DataFrame([[y.text.strip() 
               for y in x.find_all('td')] 
              for x in charts[1].find_all('tr')[1:-1]]).loc[:,1:]

ranking_df.columns = year_list

ranking_df = pd.concat([label_df, ranking_df], axis=1).set_index(['loc1', 'loc2', 'loc3'])

ranking_df = ranking_df.replace('NT',np.NaN).replace('NV',np.NaN)

for col in ranking_df.columns:
    ranking_df[col] = ranking_df[col].str.replace(r"""[A-Z]+""", '')
    ranking_df[col] = ranking_df[col].apply(lambda x: float(x))

In [11]:
ranking_df.head(5)

2014  2013  2012  \
loc1   loc2   loc3                                                          
Europe NaN    Austria Riesling, Gruner Veltliner          NaN  87.0  89.0   
       France Alsace                                      NaN  84.0  87.0   
              Bordeaux: St. Julien/Pauillac St. Estephe   NaN  78.0  87.0   
              Margaux                                     NaN  80.0  88.0   
              Graves                                      NaN  81.0  89.0   

                                                         2011  2010  2009  \
loc1   loc2   loc3                                                          
Europe NaN    Austria Riesling, Gruner Veltliner          NaN  88.0  89.0   
       France Alsace                                     85.0  79.0  82.0   
              Bordeaux: St. Julien/Pauillac St. Estephe  88.0  98.0  99.0   
              Margaux                                    87.0  95.0  97.0   
              Graves                                     86.0  99.0  98.0   

                                                         2008  2007  2006  \
loc1   loc2   loc3                                                          
Europe NaN    Austria Riesling, Gruner Veltliner         88.0  90.0  91.0   
       France Alsace                                     90.0  90.0  79.0   
              Bordeaux: St. Julien/Pauillac St. Estephe  91.0  86.0  87.0   
              Margaux                                    90.0  86.0  88.0   
              Graves                                     91.0  87.0  87.0   

                                                         2005  ...   1983  \
loc1   loc2   loc3                                             ...          
Europe NaN    Austria Riesling, Gruner Veltliner         87.0  ...    NaN   
       France Alsace                                     87.0  ...   93.0   
              Bordeaux: St. Julien/Pauillac St. Estephe  95.0  ...   86.0   
              Margaux                                    98.0  ...   95.0   
              Graves                                     96.0  ...   89.0   

                                                         1982  1981  1980  \
loc1   loc2   loc3                                                          
Europe NaN    Austria Riesling, Gruner Veltliner          NaN   NaN   NaN   
       France Alsace                                     82.0  86.0  80.0   
              Bordeaux: St. Julien/Pauillac St. Estephe  98.0  85.0  78.0   
              Margaux                                    86.0  82.0  79.0   
              Graves                                     88.0  84.0  78.0   

                                                         1979  1978  1976  \
loc1   loc2   loc3                                                          
Europe NaN    Austria Riesling, Gruner Veltliner          NaN   NaN   NaN   
       France Alsace                                     84.0  80.0  90.0   
              Bordeaux: St. Julien/Pauillac St. Estephe  85.0  87.0  84.0   
              Margaux                                    87.0  87.0  77.0   
              Graves                                     88.0  88.0  71.0   

                                                         1975  1971  1970  
loc1   loc2   loc3                                                         
Europe NaN    Austria Riesling, Gruner Veltliner          NaN   NaN   NaN  
       France Alsace                                     82.0  90.0  80.0  
              Bordeaux: St. Julien/Pauillac St. Estephe  89.0  82.0  87.0  
              Margaux                                    78.0  83.0  85.0  
              Graves                                     89.0  86.0  87.0  

[5 rows x 41 columns]

In [208]:
ranking_df.to_pickle('../priv/pkl/05_vintage_years.pkl')